<a href="https://colab.research.google.com/github/Userfound404/Video-Transformers-keras/blob/main/video_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -q https://git.io/JGc31 -O ucf101_top5.tar.gz
!tar xf ucf101_top5.tar.gz

In [3]:
!pip install -q git+https://github.com/tensorflow/docs

  Preparing metadata (setup.py) ... done


In [4]:
from tensorflow_docs.vis import embed
from tensorflow.keras import layers
from tensorflow import keras

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

In [5]:
MAX_SEQ_LENGTH = 20
NUM_FEATURES = 1024
IMG_SIZE = 128

EPOCHS = 5

This script is loading and preparing a dataset for training a video classification model using the Keras library. The script starts by loading two datasets, one for training and one for testing using the pd.read_csv function from the pandas library. The script is using a helper function load_video to load videos from a specified file path, using the OpenCV library. The script is also using a helper function crop_center to crop the center of each frame of the video to a specified image size. The script then creates a feature extractor model using the DenseNet121 architecture and pre-trained on ImageNet dataset, that will be used to extract features from each frame of the video. Then the script applies label preprocessing on the training dataset, using the StringLookup class from Keras.

The script then uses the feature extractor and the load_video function to extract features from each frame of all videos in the training dataset and store these features in an array. The script also pads shorter videos to a fixed length (MAX_SEQ_LENGTH), so that all videos have the same length. And It also shows the unique label for the training dataset

It is likely that this script is meant to be used as a part of a larger program, and that this script is only preparing the dataset for use in training a video classification model.

In [6]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")

center_crop_layer = layers.CenterCrop(IMG_SIZE, IMG_SIZE)


def crop_center(frame):
    cropped = center_crop_layer(frame[None, ...])
    cropped = cropped.numpy().squeeze()
    return cropped

Total videos for training: 594
Total videos for testing: 224


In [7]:
def load_video(path, max_frames=0):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center(frame)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

cap = cv2.VideoCapture(path): This line creates a VideoCapture object from OpenCV library, that reads the video from the specified path.

frames = []: This line initializes an empty list to store the frames of the video.

while True:: This line starts a while loop that will run until the video has been read completely.

ret, frame = cap.read(): This line reads a frame from the video, which is stored in the frame variable. The ret variable stores a Boolean value, indicating whether a frame was successfully read or not.

if not ret: break: This line checks the value of the ret variable. If ret is False, which means that no frame was read, the loop will break, and the script will proceed to the next step.

frame = crop_center(frame): This line applies the helper function crop_center() to the current frame, which crops the center of the frame to a specified image size.

frame = frame[:, :, [2, 1, 0]]: This line reorders the color channels of the frame from BGR to RGB.

frames.append(frame): This line adds the current frame to the list of frames.

if len(frames) == max_frames: break: This line checks the number of frames that have been read. If the number of frames equals to max_frames, the loop breaks.

cap.release(): This line releases the video capturing object.

return np.array(frames): This line converts the list of frames to a numpy array and returns it.